In [13]:
import os
import re
import json
from tqdm import tqdm

In [4]:
data = json.load(open('../../../transform/transformers_cache/downloads/LC-QuAD2.0-pre/dataset/test.json', 'rb'))

In [17]:
pre = Preprocess()

In [18]:
for item in tqdm(data):
    res = pre._preprocess(item)

100%|██████████| 6046/6046 [00:01<00:00, 4077.93it/s]


In [19]:
res

{'input_sparql': 'select distinct ?vr0 where { wd: manticore wdt: Theoi Project ID ?vr0 }',
 'input_process': '<extra_id_6> <extra_id_21> <extra_id_39> <extra_id_19> <extra_id_33> <extra_id_53> manticore <extra_id_54> <extra_id_59> project id <extra_id_39> <extra_id_15>',
 'target_sparql': 'select distinct ?vr0 where { wd: manticore wdt: Theoi Project ID ?vr0 }',
 'target_process': '<extra_id_6> <extra_id_21> <extra_id_39> <extra_id_19> <extra_id_33> <extra_id_53> manticore <extra_id_54> <extra_id_59> project id <extra_id_39> <extra_id_15>'}

In [16]:
class Preprocess(object):
    def __init__(self):
        
        path = './'
        ent_labels = json.load(open(os.path.join(path, 'entities.json'), 'rb'))
        rel_labels = json.load(open(os.path.join(path, 'relations.json'), 'rb'))
        
        vocab=['"', '(', 'rdfs:label', 'by', 'ask', '>', 'select', 'que', 'limit', 'jai', 'mai', 
        '?sbj', ')', 'lang', 'year', '}', '?value', 'peint', 'desc', 'where', 'ce', 'distinct', 
       'filter', 'lcase', 'order', 'la', '<', 'asc', 'en', 'contains', 'as', ',', 'strstarts', 
       '{', "'", 'j', 'count', '=', '.', '?vr0', '?vr1', '?vr2', '?vr3', '?vr4', '?vr5', '?vr6', 
       '?vr0_label', '?vr1_label', '?vr2_label', '?vr3_label', '?vr4_label', '?vr5_label', '?vr6_label',
       'wd:', 'wdt:', 'ps:', 'p:', 'pq:', '?maskvar1','null']

        vocab_dict={}
        for i,text in enumerate(vocab):
            vocab_dict[text]='<extra_id_'+str(i)+'>'

        for kk in ent_labels:
            if ent_labels[kk] is None: ent_labels[kk] = vocab_dict['null']

        self.ent_labels = ent_labels
        self.rel_labels = rel_labels
        self.vocab_dict = vocab_dict

    
    def process(self, wikisparql):
        sparql = wikisparql.replace('(',' ( ').replace(')',' ) ').replace('{',' { ')\
        .replace('}',' } ').replace(':',': ').replace(',',' , ').replace("'"," ' ")\
        .replace('.',' . ').replace('=',' = ').lower()
        sparql = ' '.join(sparql.split())
        
        _ents = re.findall( r'wd: (?:.*?) ', sparql)
        _ents_for_labels = re.findall( r'wd: (q.*?) ', sparql)
        
        _rels = re.findall( r'wdt: (?:.*?) ',sparql)
        _rels += re.findall( r' p: (?:.*?) ',sparql)
        _rels += re.findall( r' ps: (?:.*?) ',sparql)
        _rels += re.findall( r'pq: (?:.*?) ',sparql)
        
        _rels_for_labels = re.findall( r'wdt: (.*?) ',sparql)
        _rels_for_labels += re.findall( r' p: (.*?) ',sparql)
        _rels_for_labels += re.findall( r' ps: (.*?) ',sparql)
        _rels_for_labels += re.findall( r'pq: (.*?) ',sparql)

        # for j in range(len(_ents_for_labels)):
        #     if '}' in _ents[j]: 
        #         _ents[j]=''
        #     _ents[j] = _ents[j] + self.ent_labels[_ents_for_labels[j]]+' '
            
        for j in range(len(_rels_for_labels)):
            if _rels_for_labels[j] not in self.rel_labels:
                self.rel_labels[_rels_for_labels[j]] = self.vocab_dict['null']
            _rels[j] = _rels[j] + self.rel_labels[_rels_for_labels[j]]+' '
    
            
        split = sparql.split()
        for idx, item in enumerate(split):
            if item in self.ent_labels:
                split[idx] = self.ent_labels[item]
            elif item in self.rel_labels:
                split[idx] = self.rel_labels[item]

            if item in self.vocab_dict:
                split[idx] = self.vocab_dict[item]
        
        gold_query = ' '.join(split).strip()
        
        return gold_query
        
    def _preprocess(self, data):
        res = { 'input_sparql': data['input'],
                'input_process': self.process(data['input']),
                'target_sparql': data['target'],
                'target_process': self.process(data['target']),
               }

        return res

In [3]:
test_data = json.load(open('../../../transform/transformers_cache/downloads/LC-QuAD2.0-pre/dataset/test.json', 'rb'))

In [4]:
train_data = json.load(open('../../../transform/transformers_cache/downloads/LC-QuAD2.0-pre/dataset/train.json', 'rb'))

In [5]:
pre = Preprocess()

In [7]:
for item in test_data:
    res = pre._preprocess(item)

In [14]:
item

{'input': 'select ?vr0 where { ?vr0 wdt:p19 wd:q36970 . wd: wdt:p1813 ?vr0}',
 'target': 'select ?vr0 where { wd:q36970 wdt:p19 ?vr0 . ?vr0 wdt:p1813 wd:}'}

In [19]:
sparql = item['input'].replace('(',' ( ').replace(')',' ) ').replace('{',' { ')\
    .replace('}',' } ').replace(':',': ').replace(',',' , ').replace("'"," ' ")\
    .replace('.',' . ').replace('=',' = ').lower()
sparql = ' '.join(sparql.split())
sparql        

'select ?vr0 where { ?vr0 wdt: p19 wd: q36970 . wd: wdt: p1813 ?vr0 }'

In [ ]:
_ents = re.findall( r'wd: (?:.*?) ', sparql)
_ents_for_labels = re.findall( r'wd: (.*?) ', sparql)
        print(_ents)
        print(_ents_for_labels)

In [25]:
re.findall( r'wd: (?.*?) ', sparql)

error: unknown extension ?. at position 5